In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.stats import pearsonr
%matplotlib inline

In [2]:
file = pd.read_csv('desktop/ds_salary_proj/gls_jobs.csv')
file

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Data Scientist,$53K-$91K (Glassdoor est.),"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research\r\n3.8,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,$50 to $100 million (USD),-1
1,1,Healthcare Data Scientist,$63K-$112K (Glassdoor est.),What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System\r\n3.4,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),-1
2,2,Data Scientist,$80K-$90K (Glassdoor est.),"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4\r\n4.8,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,Security Services,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$56K-$97K (Glassdoor est.),*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL\r\n3.8,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,Energy,"Oil, Gas, Energy & Utilities",$500 million to $1 billion (USD),"Oak Ridge National Laboratory, National Renewa..."
4,4,Data Scientist,$86K-$143K (Glassdoor est.),Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions\r\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951,951,Senior Data Engineer,$72K-$133K (Glassdoor est.),THE CHALLENGE\r\nEventbrite has a world-class ...,4.4,Eventbrite\r\n4.4,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,Internet,Information Technology,$100 to $500 million (USD),"See Tickets, TicketWeb, Vendini"
952,952,"Project Scientist - Auton Lab, Robotics Institute",$56K-$91K (Glassdoor est.),The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute\r\n2.6,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,Colleges & Universities,Education,Unknown / Non-Applicable,-1
953,953,Data Science Manager,$95K-$160K (Glassdoor est.),Data Science ManagerResponsibilities:\r\n\r\nO...,3.2,"Numeric, LLC\r\n3.2","Allentown, PA","Chadds Ford, PA",1 to 50 employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$5 to $10 million (USD),-1
954,954,Data Engineer,-1,Loading...\r\n\r\nTitle: Data Engineer\r\n\r\n...,4.8,IGNW\r\n4.8,"Austin, TX","Portland, OR",201 to 500 employees,2015,Company - Private,IT Services,Information Technology,$25 to $50 million (USD),Slalom


In [3]:
#test.columns
file = file.drop('Unnamed: 0',axis=1)
#Salary Parsing
file = file[file['Salary Estimate']!='-1']
file.loc[:,'Salary Estimate'] = file.loc[:,'Salary Estimate'].apply(lambda x: str(x.split('(')[0]).replace('K',"").replace('$',""))
left_parse = file['Salary Estimate'].apply(lambda x: ''.join(filter(str.isdigit,x.split('-')[0])))
right_parse = file['Salary Estimate'].apply(lambda x: ''.join(filter(str.isdigit,x.split('-')[1])))
salary_filtered = left_parse + "-" + right_parse
file['Salary Estimate'] = salary_filtered 
file['min_salary'] = left_parse.apply(lambda x: int(x))
file['max_salary'] = right_parse.apply(lambda x: int(x))
file['avg_salary'] = (file['min_salary']+file['max_salary'])/2
#Company Name Text Only
file['Company Name'] = file['Company Name'].apply(lambda x: x.replace("\r\n"," "))
file['Company txt'] = file.apply(lambda x: x['Company Name'] if x['Rating']<0 else x['Company Name'][:-3], axis=1)
#State field
file['Job_State'] = file['Location'].apply(lambda x: x[-2:])
#Age of company
file = file[file['Founded']!= -1]
file.loc[:,'Age of company'] = 2021-file['Founded']
#Parsing job description
file['Python'] = file['Job Description'].apply(lambda x: True if "python" in x.lower() else False)
file['R-Studio'] = file['Job Description'].apply(lambda x: True if "r-studio" or "r studio" in x.lower() else False)
file['Spark'] = file['Job Description'].apply(lambda x: True if "spark" in x.lower() else False)
file['Excel'] = file['Job Description'].apply(lambda x: True if "excel" in x.lower() else False)
file['Medical'] = file['Job Description'].apply(lambda x: True if "medical" in x.lower() else False)
def jobsearch(string_jt):
    str_id = string_jt.lower()
    res = ""
    job_list = ["python","r-studio","r studio","spark","excel","medical",]
    for i in job_list:
        if i in str_id:
            res += i
            res += ","
    if res=="":
        res = "None"
    return res.strip(",")
file['Job_toolkit'] = file['Job Description'].apply(jobsearch)

In [7]:
cleaned_data = file
cleaned_data.to_csv('Cleaned_Salary_Data_modified',index=False)

In [5]:
def title_simplifier(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'jr'
    else:
        return 'na'
#Job description length
file['desc_len'] = file['Job Description'].apply(lambda x: len(x))
#Competitor count
file['Comp_count'] = file['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)
#Company Name modification
file.loc[:,'Company Name'] = file['Company Name'].apply(lambda x: (' ').join(x.split()[:-1]))
#
file['Job_Hierarchy'] = file['Job Title'].apply(seniority)
file['Job_Simp'] = file['Job Title'].apply(title_simplifier)

In [6]:
file

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,...,Python,R-Studio,Spark,Excel,Medical,Job_toolkit,desc_len,Comp_count,Job_Hierarchy,Job_Simp
0,Data Scientist,53-91,"Data Scientist\r\nLocation: Albuquerque, NM\r\...",3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,Company - Private,...,True,True,False,True,False,"python,excel",2555,0,na,data scientist
1,Healthcare Data Scientist,63-112,What You Will Do:\r\n\r\nI. General Summary\r\...,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,Other Organization,...,True,True,False,False,True,"python,medical",4828,0,na,data scientist
2,Data Scientist,80-90,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,Company - Private,...,True,True,True,True,False,"python,spark,excel",3495,0,na,data scientist
3,Data Scientist,56-97,*Organization and Job ID**\r\nJob ID: 310709\r...,3.8,PNNL,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,Government,...,True,True,False,False,False,python,3926,3,na,data scientist
4,Data Scientist,86-143,Data Scientist\r\nAffinity Solutions / Marketi...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,...,True,True,False,True,False,"python,excel",2748,3,na,data scientist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
949,"Principal, Data Science - Advanced Analytics",86-137,IQVIA is the leading human data science compan...,3.6,IQVIA,"Plymouth Meeting, PA","Durham, NC",10000+ employees,2017,Company - Public,...,False,True,False,False,True,medical,5064,3,senior,na
950,"Sr Scientist, Immuno-Oncology - Oncology",58-111,Site Name: USA - Massachusetts - Cambridge\r\n...,3.9,GSK,"Cambridge, MA","Brentford, United Kingdom",10000+ employees,1830,Company - Public,...,False,True,False,False,False,None,6219,3,senior,na
951,Senior Data Engineer,72-133,THE CHALLENGE\r\nEventbrite has a world-class ...,4.4,Eventbrite,"Nashville, TN","San Francisco, CA",1001 to 5000 employees,2006,Company - Public,...,True,True,True,False,True,"python,spark,medical",6167,3,senior,data engineer
952,"Project Scientist - Auton Lab, Robotics Institute",56-91,The Auton Lab at Carnegie Mellon University is...,2.6,Software Engineering Institute,"Pittsburgh, PA","Pittsburgh, PA",501 to 1000 employees,1984,College / University,...,False,True,False,True,False,excel,3107,0,na,na
